<a href="https://colab.research.google.com/github/zeynepsenatatli/MachineLearningExercises/blob/main/UE08_C7E9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dataset and Classifiers (from previous question Chapter 7 Exercise 8)

In [ ]:
from tensorflow import keras
import numpy as np

In [ ]:
# Load the MNIST dataset
mnist = keras.datasets.mnist
(x_train_full, y_train_full), (x_test, y_test) = mnist.load_data()

x_train_full = x_train_full / 255
x_train_full = np.reshape(x_train_full, (x_train_full.shape[0], 28 * 28))

x_valid, x_train = x_train_full[:10000], x_train_full[10000:]
y_valid, y_train = y_train_full[:10000], y_train_full[10000:]

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train.shape

(50000, 784)

In [ ]:
x_test = x_test / 255
x_test = np.reshape(x_test, (x_test.shape[0], 28 * 28))

In [ ]:
x_test.shape

(10000, 784)

Then train various classifiers, such as a random forest classifier, an extra-trees classifier, and an SVM classifier.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(x_train, y_train)

etc = ExtraTreesClassifier(n_estimators=100, random_state=42)
etc.fit(x_train, y_train)

ExtraTreesClassifier(random_state=42)

In [ ]:
svm = SVC(kernel='linear', C=0.1, random_state=42)
svm.fit(x_train, y_train)

SVC(C=0.1, kernel='linear', random_state=42)

In [ ]:
mlp = MLPClassifier(random_state=42)
mlp.fit(x_train, y_train)

MLPClassifier(random_state=42)

In [ ]:
classifiers = [('Random Forest Classifier', rfc),
               ('Extra-Trees Classifier', etc),
               ('Linear SVM Classifier', svm),
               ('Multi Layer Perceptron', mlp)
]

### Stacking Ensemble

Run the individual classifiers from the previous exercise to make predictions on the validation set, and create a new training set with the resulting predictions: each training instance is a vector containing the set of predictions from all your classifiers for an image, and the target is the image's class. Train a classifier on this new training set.

In [ ]:
estimators = [rfc, etc, svm, mlp]

In [ ]:
X_valid_predictions = np.empty((len(x_valid), len(estimators)), dtype=object)

X_valid_predictions.shape

(10000, 4)

In [ ]:
for index, estimator in enumerate(estimators):
    X_valid_predictions[:, index] = estimator.predict(x_valid)

X_valid_predictions

array([[5, 5, 5, 5],
       [0, 0, 0, 0],
       [4, 4, 4, 4],
       ...,
       [6, 6, 6, 6],
       [9, 9, 9, 9],
       [7, 7, 7, 7]], dtype=object)

In [ ]:
rnd_forest = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest.fit(X_valid_predictions, y_valid)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [ ]:
rnd_forest.oob_score_

0.9782

Congratulations, you have just trained a blender, and together with the classifiers they form a stacking ensemble! Now let's evaluate the ensemble on the test set. For each image in the test set, make predictions with all your classifiers, then feed the predictions to the blender to get the ensemble's predictions. How does it compare to the voting classifier you trained earlier?

In [ ]:
X_test_predictions = np.empty((len(x_test), len(estimators)), dtype=object)

In [ ]:
X_test_predictions.shape

(10000, 4)

In [ ]:
for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(x_test)

X_test_predictions

array([[7, 7, 7, 7],
       [2, 2, 2, 2],
       [1, 1, 1, 1],
       ...,
       [4, 4, 4, 4],
       [5, 5, 5, 5],
       [6, 6, 6, 6]], dtype=object)

In [ ]:
y_pred = rnd_forest.predict(X_test_predictions)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.9757

The accuracy of voting classifier (from previous question) was 0.9705. This (stacking ensemble) is slightly better.


---

Now try again using a StackingClassifier instead: do you get better performance? If so, why?

*Since StackingClassifier uses K-Fold cross-validation, we don't need a separate validation set, so let's join the training set and the validation set into a bigger training set*

In [ ]:
x_train_full.shape

(60000, 784)

In [ ]:
y_train_full.shape

(60000,)

*Stacking Classifier*

In [ ]:
from sklearn.ensemble import StackingClassifier

In [ ]:
stack_clf = StackingClassifier(classifiers, final_estimator=rnd_forest)
stack_clf.fit(x_train_full, y_train_full)

StackingClassifier(estimators=[('Random Forest Classifier',
                                RandomForestClassifier(random_state=42)),
                               ('Extra-Trees Classifier',
                                ExtraTreesClassifier(random_state=42)),
                               ('Linear SVM Classifier',
                                SVC(C=0.1, kernel='linear', random_state=42)),
                               ('Multi Layer Perceptron',
                                MLPClassifier(random_state=42))],
                   final_estimator=RandomForestClassifier(n_estimators=200,
                                                          oob_score=True,
                                                          random_state=42))

In [ ]:
stack_clf.score(x_test, y_test)

0.9815